<a href="https://colab.research.google.com/github/PaulFSherwood/python/blob/master/AI/NaturalLanguageProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bag of Words**

In [59]:
vocab = {}  # maps word to integer representing it
word_encoding = 1
def bag_of_words(text):
  global word_encoding

  words = text.lower().split(" ")  # create a list of all of the words in the text, well assume there is no grammar in our text for this example
  bag = {}  # stores all of the encodings and their frequency

  for word in words:
    if word in vocab:
      encoding = vocab[word]  # get encoding from vocab
    else:
      vocab[word] = word_encoding
      encoding = word_encoding
      word_encoding += 1
    
    if encoding in bag:
      bag[encoding] += 1
    else:
      bag[encoding] = 1
  
  return bag

text = "this is a test to see if this test will work is is test a a"
bag = bag_of_words(text)
print(bag)
print(vocab)

{1: 2, 2: 3, 3: 3, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


In [60]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_bag = bag_of_words(positive_review)
neg_bag = bag_of_words(negative_review)

print("Positive:", pos_bag)
print("Negative:", neg_bag)

Positive: {10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 1, 5: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1}
Negative: {10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 1, 5: 1, 16: 1, 21: 1, 18: 1, 19: 1, 20: 1, 17: 1}


# **Integer Encoding**

In [61]:
vocab = {}  
word_encoding = 1
def one_hot_encoding(text):
  global word_encoding

  words = text.lower().split(" ") 
  encoding = []  

  for word in words:
    if word in vocab:
      code = vocab[word]  
      encoding.append(code) 
    else:
      vocab[word] = word_encoding
      encoding.append(word_encoding)
      word_encoding += 1
  
  return encoding

text = "this is a test to see if this test will work is is test a a"
encoding = one_hot_encoding(text)
print(encoding)
print(vocab)

[1, 2, 3, 4, 5, 6, 7, 1, 4, 8, 9, 2, 2, 4, 3, 3]
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


In [62]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_encode = one_hot_encoding(positive_review)
neg_encode = one_hot_encoding(negative_review)

print("Positive:", pos_encode)
print("Negative:", neg_encode)

Positive: [10, 11, 12, 13, 14, 15, 5, 16, 17, 18, 19, 14, 20, 21]
Negative: [10, 11, 12, 13, 14, 15, 5, 16, 21, 18, 19, 14, 20, 17]


# **Recurrent Neural Networks (RNN's)**
**LSTM**

In [63]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [64]:
# Lets look at one review
train_data[1]

[1,
 194,
 1153,
 194,
 8255,
 78,
 228,
 5,
 6,
 1463,
 4369,
 5012,
 134,
 26,
 4,
 715,
 8,
 118,
 1634,
 14,
 394,
 20,
 13,
 119,
 954,
 189,
 102,
 5,
 207,
 110,
 3103,
 21,
 14,
 69,
 188,
 8,
 30,
 23,
 7,
 4,
 249,
 126,
 93,
 4,
 114,
 9,
 2300,
 1523,
 5,
 647,
 4,
 116,
 9,
 35,
 8163,
 4,
 229,
 9,
 340,
 1322,
 4,
 118,
 9,
 4,
 130,
 4901,
 19,
 4,
 1002,
 5,
 89,
 29,
 952,
 46,
 37,
 4,
 455,
 9,
 45,
 43,
 38,
 1543,
 1905,
 398,
 4,
 1649,
 26,
 6853,
 5,
 163,
 11,
 3215,
 10156,
 4,
 1153,
 9,
 194,
 775,
 7,
 8255,
 11596,
 349,
 2637,
 148,
 605,
 15358,
 8003,
 15,
 123,
 125,
 68,
 23141,
 6853,
 15,
 349,
 165,
 4362,
 98,
 5,
 4,
 228,
 9,
 43,
 36893,
 1157,
 15,
 299,
 120,
 5,
 120,
 174,
 11,
 220,
 175,
 136,
 50,
 9,
 4373,
 228,
 8255,
 5,
 25249,
 656,
 245,
 2350,
 5,
 4,
 9837,
 131,
 152,
 491,
 18,
 46151,
 32,
 7464,
 1212,
 14,
 9,
 6,
 371,
 78,
 22,
 625,
 64,
 1382,
 9,
 8,
 168,
 145,
 23,
 4,
 1690,
 15,
 16,
 4,
 1355,
 5,
 28,
 6,
 52,
 

In [65]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)
# train_data[1]

# **Creating the Model**

In [66]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [67]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 32)          2834688   
_________________________________________________________________
lstm_4 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


**Training**

In [68]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 18s 26ms/step - loss: 0.5275 - acc: 0.7342 - val_loss: 0.3228 - val_acc: 0.8692
Epoch 2/10
625/625 [==============================] - 16s 25ms/step - loss: 0.2394 - acc: 0.9103 - val_loss: 0.2679 - val_acc: 0.8904
Epoch 3/10
625/625 [==============================] - 16s 25ms/step - loss: 0.1810 - acc: 0.9344 - val_loss: 0.2696 - val_acc: 0.8914
Epoch 4/10
625/625 [==============================] - 16s 25ms/step - loss: 0.1441 - acc: 0.9482 - val_loss: 0.2966 - val_acc: 0.8932
Epoch 5/10
625/625 [==============================] - 16s 25ms/step - loss: 0.1253 - acc: 0.9590 - val_loss: 0.2921 - val_acc: 0.8882
Epoch 6/10
625/625 [==============================] - 16s 25ms/step - loss: 0.1102 - acc: 0.9628 - val_loss: 0.2996 - val_acc: 0.8796
Epoch 7/10
625/625 [==============================] - 16s 25ms/step - loss: 0.0941 - acc: 0.9696 - val_loss: 0.3970 - val_acc: 0.8654
Epoch 8/10
625/625 [==============================] - 16s 25ms

In [69]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 5s 7ms/step - loss: 0.4820 - acc: 0.8474
[0.4819587171077728, 0.8473600149154663]


**Making Predictions**

In [70]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)


[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4

In [71]:
#print(word_index)
# for bacon in word_index:
#     print(bacon)

In [72]:
# while were at it lets make a decode function

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]
  
print(decode_integers(encoded))

that movie was just amazing so amazing


In [73]:
# now time to make a prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred) 
  print(result[0])

positive_review = "That movie was great amazing awesome! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)


[0.82697695]
[0.20299049]


# **RNN Play Generator**

In [74]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [75]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# **Loading Your Own Data**

In [76]:
# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

**Read Contents of File**

In [77]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [78]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



**Encoding**

In [79]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [80]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [81]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


**Creating Training Examples**

In [82]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [83]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [84]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [85]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [86]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Building the Model

In [87]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
lstm_5 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_5 (Dense)              (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


# Creating a Loss Function

In [88]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [89]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-1.62505324e-03  3.59386881e-03 -6.65399421e-06 ... -1.15153252e-03
   -6.55478612e-03 -3.47776012e-03]
  [-5.48006082e-03  6.02463307e-03 -9.94449016e-04 ...  1.77889329e-03
    1.64334627e-03 -3.04601551e-03]
  [-1.40967348e-03  4.03190730e-03  3.88384052e-03 ...  1.12839509e-04
    2.53246399e-04 -3.50410584e-03]
  ...
  [-2.28423253e-03  2.12398381e-03  4.57549980e-03 ...  3.09996936e-03
   -1.15486942e-02 -1.62990019e-03]
  [-4.38899733e-03  4.94328979e-03  3.64788598e-03 ...  1.30360923e-03
   -1.55024203e-02 -3.20083252e-03]
  [-2.02452950e-03 -8.09304067e-04  1.01950183e-03 ...  5.02040423e-03
   -6.65466907e-03 -2.85698520e-03]]

 [[ 2.85359588e-03 -1.08860189e-03  3.80810164e-03 ... -9.20211198e-04
   -2.46711599e-04 -1.54756568e-03]
  [-1.09100039e-03  5.52108977e-03 -1.48369896e-03 ...  1.81643502e-03
   -4.14534938e-03  3.33285891e-03]
  [-3.05662001e-03  5.62557532e-03 -8.21529306e-04 ... -5.21969749e-03
   -6.85834046e-03  4.90637030e-04]
  ...
  [-8.129

In [90]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-1.6250532e-03  3.5938688e-03 -6.6539942e-06 ... -1.1515325e-03
  -6.5547861e-03 -3.4777601e-03]
 [-5.4800608e-03  6.0246331e-03 -9.9444902e-04 ...  1.7788933e-03
   1.6433463e-03 -3.0460155e-03]
 [-1.4096735e-03  4.0319073e-03  3.8838405e-03 ...  1.1283951e-04
   2.5324640e-04 -3.5041058e-03]
 ...
 [-2.2842325e-03  2.1239838e-03  4.5754998e-03 ...  3.0999694e-03
  -1.1548694e-02 -1.6299002e-03]
 [-4.3889973e-03  4.9432898e-03  3.6478860e-03 ...  1.3036092e-03
  -1.5502420e-02 -3.2008325e-03]
 [-2.0245295e-03 -8.0930407e-04  1.0195018e-03 ...  5.0204042e-03
  -6.6546691e-03 -2.8569852e-03]], shape=(100, 65), dtype=float32)


In [91]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-1.6250532e-03  3.5938688e-03 -6.6539942e-06  5.9443144e-03
  1.5555980e-03 -6.2354421e-04 -3.1057408e-03 -1.0487026e-03
 -1.2660190e-03  3.3148478e-03 -4.7197449e-03 -9.3634659e-04
  2.1885730e-04 -2.7516764e-03  1.7832371e-04  4.4216635e-03
 -8.6925516e-04  5.4131807e-03 -3.5724503e-03  1.4334418e-03
  6.0079182e-03 -7.3309750e-03  1.7004315e-03  1.2101134e-04
  8.2305754e-03 -2.2327718e-03 -9.2433882e-04  3.2517659e-03
 -7.8073703e-05 -7.8498921e-04 -5.2477373e-04 -1.2755372e-03
 -8.6616967e-03 -3.9207037e-03 -1.7413723e-03 -3.4206070e-03
  3.1163287e-04  1.5060733e-03 -9.3835348e-04  3.5825623e-03
  4.2889547e-03  3.4167967e-03  8.4788958e-03  3.6620616e-03
 -7.5466279e-04 -1.3324963e-03  3.3596891e-03 -3.9279519e-04
  1.9162174e-03 -1.1268323e-03 -2.0255223e-03  1.0944073e-02
  2.4563367e-03 -5.0793483e-04 -1.2503150e-03 -2.9480830e-04
  1.9051391e-03  1.1698531e-03 -4.3812487e-03  2.4237712e-03
 -4.1134600e-03  8.1277732e-04 -1.1515325e-03 -6.5547861e-03
 -3.477760

In [92]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"h?DsjjMH3SVsHDXM$PvSWJojMP!BOUkBMTB N-VTkwsUOwbMwxahmkSABYq-.sTXwASQ$j.sPKkJ3Ns!AtsdyMaWY:il?V'rvZtE"

In [93]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

**Compiling the Model**

In [94]:
model.compile(optimizer='adam', loss=loss)

**Creating Checkpoints**

In [95]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

**Training**

In [96]:
history = model.fit(data, epochs=150, callbacks=[checkpoint_callback])

Epoch 1/150
172/172 [==============================] - 16s 80ms/step - loss: 3.0371
Epoch 2/150
172/172 [==============================] - 15s 79ms/step - loss: 1.9738
Epoch 3/150
172/172 [==============================] - 14s 76ms/step - loss: 1.6797
Epoch 4/150
172/172 [==============================] - 14s 75ms/step - loss: 1.5196
Epoch 5/150
172/172 [==============================] - 14s 75ms/step - loss: 1.4312
Epoch 6/150
172/172 [==============================] - 14s 76ms/step - loss: 1.3679
Epoch 7/150
172/172 [==============================] - 14s 77ms/step - loss: 1.3174
Epoch 8/150
172/172 [==============================] - 14s 76ms/step - loss: 1.2760
Epoch 9/150
172/172 [==============================] - 14s 76ms/step - loss: 1.2378
Epoch 10/150
172/172 [==============================] - 14s 75ms/step - loss: 1.2017
Epoch 11/150
172/172 [==============================] - 14s 76ms/step - loss: 1.1607
Epoch 12/150
172/172 [==============================] - 14s 76ms/step - lo

**Loading the Model**

In [97]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [98]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [100]:
# checkpoint_num = 10
# model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
# model.build(tf.TensorShape([1, None]))

**Generating Text**

In [101]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [103]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: Frank
Franks that fell, have not
My guiled all for willing.

DUCHESS OF YORK:
I will be slanderets to my demand.

ROMEO:
Would all the ruin of doods!
What, do I thank thee, Is thy news from her, hence can help from you to Angelo.
Now when the sea was a man-child
make you, good night; and therefore part the Lady Valeria.
But now I see; or if you be much resolved
You may be heard that but man die the one
Where I was nothing.

DION:
If they can curse them all my heart to see this captives;
The dukedom yet have bleeding thee,
For but of my report.

COMINIUS:
Who shall hath prevent them, and they shall straight
five wondering: or, if lions drop is lessor themselves? what, what?--
With all god prick mon's charge?
Telling this man that great dewren here do fight.

MERCUTIO:
Nay, and for that sin
Out three h
